In [4]:
### Main cell - this needs to be run last generally, but has been put here for simplicity
# The function takes 4 inputs: The directory of the image to find clothes based on, the image-emotion CNN, the image-mediatype CNN, and the feature-style CNN
# If the CNNs are not provided, they default to the locations shown
predict_image_style(  imgdir = "/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/BAM/images-annotated/images-annotated/0000/1220000.jpg",
                      emote_predictor = Predictor("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/BAM/models/2022-07-13/prototype-emotion-1"),
                      media_predictor = Predictor("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/BAM/models/2022-07-13/prototype-media-1"),
                      style_predictor = Predictor("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/HipsterWars/models/widths = 1024,2048,128;activations = relu,relu,relu;lrate = 0.0001;epochs = 50"),
                      prediction_weightings = 0.1)

Current best image found: 1 with similarity of 1.4445382157118654
Current best image found: 45 with similarity of 1.4430963857118653
Current best image found: 126 with similarity of 1.4423692057118656
Current best image found: 143 with similarity of 1.4044606101506856
Current best image found: 216 with similarity of 1.342028685150685
Current best image found: 239 with similarity of 1.3033149728109406
Current best image found: 300 with similarity of 1.293730185662923


300

In [2]:
## Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
### Functions needed for the predict_image_style function to work

import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image

class Predictor:
  def __init__(self,model_dir):
    self.model_dir = model_dir
    self.model = tf.keras.models.load_model(model_dir)
    p_start = model_dir.rfind("/")+1
    self.model_name = model_dir[p_start:]
    return

# Take an input image directory and find the most similar clothing results using both predicted and true outputs
def predict_image_style(imgdir,
                        emote_predictor = Predictor("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/BAM/models/2022-07-13/prototype-emotion-1"),
                        media_predictor = Predictor("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/BAM/models/2022-07-13/prototype-media-1"),
                        style_predictor = Predictor("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/HipsterWars/models/widths = 1024,2048,128;activations = relu,relu,relu;lrate = 0.0001;epochs = 50"),
                        prediction_weightings = 0.5):
  # Predict the style for this image
  feature_tens = predict_features([imgdir],emote_predictor,media_predictor)
  style_tens = style_predictor.model.predict(feature_tens)
  ## Get the actual and predicted styles of known clothes
  styles_actual = get_hipsterdata()
  styles_predicted_dir = "/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/HipsterWars/clothing_predictions/" + \
                        f"emote_predictor,{emote_predictor.model_name};media_predictor,{media_predictor.model_name};style_predictor,{style_predictor.model_name}.csv"
  styles_predicted = pd.read_csv(styles_predicted_dir)
  # Create a combined dataframe
  actual_outs,actual_ids,pred_outs,pred_ids = list(styles_actual["style"]),list(styles_actual["ID"]),list(styles_predicted["styles"]),list(styles_predicted["ID"])
  # Sort both by ID
  actual_outs = [x for _,x in sorted(zip(actual_ids,actual_outs))]
  actual_ids = sorted(actual_ids)
  pred_outs = [x for _,x in sorted(zip(pred_ids,pred_outs))]
  del pred_ids
  # Start scoring them all by similarity and constantly print the best one
  best_score,bsi = np.inf,-1
  for i in range(len(actual_outs)):
    # Convert the pred_outs[i] string to a list
    po = pred_outs[i]
    po = po.split(" ")
    for j in range(len(po)):
      po[j] = po[j].replace("[","")
      po[j] = po[j].replace("]","")
    j = 0
    while(j<len(po)):
      if(type(po[j])==float):
        continue
      if(len(po[j])==0):
        po.pop(j)
        continue
      try:
        po[j] = float(po[j])
        j += 1
      except:
        po.pop(i)
    res = compare_to_styles(style_tens[0],actual_outs[i],po,pred_weight = prediction_weightings)
    # If the new result is more similar than the current best, report this, and update the best score + best score index
    if(res<=best_score):
      print(f"Current best clothing found: ID {actual_ids[i]} with similarity of {res}")
      best_score = res
      bsi = i
  return actual_ids[bsi]

# Compare an image output to known clothing styles and style predictions
def compare_to_styles(image_style_preds,true_styles,predicted_styles=False,pred_weight = 0.5):
  sim = 0.0
  for i in range(len(image_style_preds)):
    sim += abs(image_style_preds[i]-float(true_styles[i]))
    # If the predictions of this clothing image is known, add the differences here. If not, add the 'pred_weight'
    if(type(predicted_styles)!=bool):
      sim += pred_weight * abs(image_style_preds[i]-float(predicted_styles[i]))
    else:
      sim += pred_weight
  return sim

# Get prediction for emotion types
def predict_emotion(imgtens,predictor):
  hipster_emote_preds = predictor.model.predict(imgtens)
  return pd.DataFrame(data=hipster_emote_preds,columns=["prob_peaceful","prob_happy","prob_gloomy","prob_scary"])


# Get predictions for media types
def predict_media(imgtens,predictor):
  hipster_media_preds = predictor.model.predict(imgtens)
  return pd.DataFrame(data=hipster_media_preds,columns=["prob_pen","prob_oilpaint","prob_watercolor","prob_comic","prob_graphite","prob_vectorart","prob_3d"])

def predict_features(imgdirs,
                     emote_predictor = Predictor("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/BAM/models/2022-07-13/prototype-emotion-1"),
                     media_predictor = Predictor("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/BAM/models/2022-07-13/prototype-media-1"),
                     emote_save_dir = False,
                     media_save_dir = False):
  # Convert the image directories to tensors
  tens = dirs_to_tensors(imgdirs)
  # Predict this list of tensors
  emote_pred = predict_emotion(tens,emote_predictor)
  media_pred = predict_media(tens,media_predictor)
  # Delete the initial tensors
  del tens
  # Save the dataframe if appropriate
  if(type(emote_save_dir)==str):
    with open(emote_save_dir,"w") as f:
      emote_pred.to_csv(f,index=False,line_terminator="\n")
  if(type(media_save_dir)==str):
    with open(media_save_dir,"w") as f:
      media_pred.to_csv(f,index=False,line_terminator="\n")
  # Convert the predictions to single 11-column tensors
  tens = []
  for ind in emote_pred.index:
    tens.append(tf.convert_to_tensor(list(emote_pred.iloc[ind])+list(media_pred.iloc[ind])))
  tens = tf.convert_to_tensor(tens)
  return tens

## Create a dataframe with feature outputs and clothing styles
def get_hipsterdata(attribute_types = ["emote","media"]):
  # Get the image directories and CNN outputs
  data = combine_preds(attribute_types)
  # Add the image ids and styles
  clothing_csv = pd.read_csv(open("/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/HipsterWars/hipster_to_csv_test.csv","r"))
  labels = list(clothing_csv["Label"])
  # Convert the directories to ids
  dirids = list(data["image_directory"])
  ids,nums = [],{}
  for i in range(10):
    nums[str(i)] = True
  for dirid in dirids:
    # Get the first number in the string
    numdec = len(dirid)-5
    while(dirid[numdec-1] in nums):
      numdec -= 1
    # Append the id only
    ids.append(int(dirid[numdec:-4]))
  # Add the IDs to the original data
  data["ID"] = ids
  del data["image_directory"]
  # Get the styles for each value
  styles = []
  styledict = {"Hipster":0,"Goth":1,"Preppy":2,"Pinup":3,"Bohemian":4}
  for tempid in ids:
    temp = clothing_csv.loc[clothing_csv["ID"]==tempid]
    label = list(temp["Label"])[0]
    labellist = [0 for i in range(len(styledict))]
    labellist[styledict[label]] += 1
    label = tf.convert_to_tensor(labellist,dtype=float)
    styles.append(label)
  data["style"] = styles
  return data

## Take HipsterWars predictions and combine them into single tensors
def combine_preds(types=["emote","media"]):
  framelocs = {"emote":"/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/HipsterWars/emotion_predictions.csv","media":"/content/drive/MyDrive/Data-Science/Art-to-fashion/Data/HipsterWars/media_predictions.csv"}
  frames = []
  for frameloc in types:
    frames.append(pd.read_csv(framelocs[frameloc]))
  imdirs = frames[0]["image_directory"]
  outs = []
  for frame in frames:
    for col in frame.columns:
      if(col=="image_directory"):
        continue
      outs.append(list(frame[col]))
  # Invert the lists obtained
  outputs = [list(i) for i in zip(*outs)]
  del outs
  # Convert the outputs to tensors
  for i in range(len(outputs)):
    outputs[i] = tf.convert_to_tensor(outputs[i],dtype=float)
  # Convert to dataframe
  return pd.DataFrame(data=list(zip(imdirs,outputs)),columns = ["image_directory","output"])

# Convert a series of image directories to tensors
def dirs_to_tensors(dirlist):
  imglist = [load_img(imdir) for imdir in dirlist]
  tenlist = [tf.convert_to_tensor(x) for x in imglist]
  del imglist
  inten = tf.convert_to_tensor(tenlist)
  del tenlist
  return inten

# Load image, taken from BAM attributions.py
def load_img(fdir,model_shape=(224,224)):
  img = Image.open(tf.io.gfile.GFile(fdir, 'rb'))
  img = preprocess_img(img,model_shape)
  return img

# Preprocessing single image; main part is resizing the image to 224x224x3
def preprocess_img(img,model_shape=(224,224),_means=[123.68, 116.78, 103.94]):
    img = img.convert('RGB').resize(model_shape, Image.BILINEAR)
    channel_means = np.expand_dims(np.expand_dims(_means, 0), 0)
    img_arr = np.array(img, dtype=np.float32) - channel_means.astype(np.float32)
    return img_arr